# Text Generation Project


The runtime on several of these cells is becoming wayyy too long due to the size of the dataset.  We'll have to transition the notebook over to google colab.

## Abstract

<b> Write an abstract here, dude. </b>

## Introduction

In this project, we'll be building a text generator that will be able to generate fake Amazon product reviews that will, hopefully, be indistinguishable from normal, everyday reviews left by actual human beings.  This is, by no means, meant to be for fraudulent uses.  This is an excercise in generating believable text and we were fortunate enough to have a large dataset of Amazon reviews to experiement with. 

### Imports

In [1]:
# The usuals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
import re

# NLTK stuff
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# Tensorflow stuff
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint




Bad key "text.kerning_factor" on line 4 in
/Users/spags/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### Functions

In [2]:
# NLP Functions

def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r"", text)


def remove_n(text):
    n = re.compile(r'\n')
    return n.sub(r'', text)


def remove_emoji(string):
    emoji_pattern = re.compile(pattern = 
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "]+",
    flags = re.UNICODE
    )
    return emoji_pattern.sub(r"", string)


def clean_text(text_column):
    text_column = text_column.apply(lambda x: remove_url(x))
    text_column = text_column.apply(lambda x: remove_n(x))
    text_column = text_column.apply(lambda x: remove_emoji(x))
    return text_column

## Obtain

In [3]:
appliances_df = pd.read_json('Appliances.json', lines = True)
print(appliances_df.shape)
appliances_df.head()

(602777, 12)


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,2,False,"11 27, 2013",A3NHUQ33CFH3VM,1118461304,{'Format:': ' Hardcover'},Greeny,Not one thing in this book seemed an obvious o...,Clear on what leads to innovation,1385510400,NaN
1,5,NaN,False,"11 1, 2013",A3SK6VNBQDNBJE,1118461304,{'Format:': ' Kindle Edition'},Leif C. Ulstrup,I have enjoyed Dr. Alan Gregerman's weekly blo...,Becoming more innovative by opening yourself t...,1383264000,NaN
2,5,NaN,False,"10 10, 2013",A3SOFHUR27FO3K,1118461304,{'Format:': ' Hardcover'},Harry Gilbert Miller III,Alan Gregerman believes that innovation comes ...,The World from Different Perspectives,1381363200,NaN
3,5,NaN,False,"10 9, 2013",A1HOG1PYCAE157,1118461304,{'Format:': ' Hardcover'},Rebecca Ripley,"Alan Gregerman is a smart, funny, entertaining...",Strangers are Your New Best Friends,1381276800,NaN
4,5,10,False,"09 7, 2013",A26JGAM6GZMM4V,1118461304,{'Format:': ' Hardcover'},Robert Morris,"As I began to read this book, I was again remi...","How and why it is imperative to engage, learn ...",1378512000,NaN


In [4]:
# Checking for null values....since we can see them already in several columns

appliances_df.isna().sum()

overall                0
vote              537515
verified               0
reviewTime             0
reviewerID             0
asin                   0
style             464804
reviewerName          15
reviewText           324
summary              128
unixReviewTime         0
image             593519
dtype: int64

In [5]:
# We can get rid of most of these columns since they aren't relevant to this project

drop_columns = ['overall', 'vote', 'verified', 'reviewTime', 'reviewerID',
                'asin', 'style', 'unixReviewTime', 'image', 'reviewerName']

appliances_df.drop(drop_columns, axis = 1, inplace = True)

print(appliances_df.shape)
appliances_df.head()

(602777, 2)


,reviewText,summary
0,Not one thing in this book seemed an obvious o...,Clear on what leads to innovation
1,I have enjoyed Dr. Alan Gregerman's weekly blo...,Becoming more innovative by opening yourself t...
2,Alan Gregerman believes that innovation comes ...,The World from Different Perspectives
3,"Alan Gregerman is a smart, funny, entertaining...",Strangers are Your New Best Friends
4,"As I began to read this book, I was again remi...","How and why it is imperative to engage, learn ..."


#### Summary Column

In [6]:
# for i in appliances_df['summary']:
#     print(i)

> Not really sure what to do with the summary column just yet, so we'll leave this as-is for now. 

## Scrubbing

### Null Values

In [7]:
# Let's check the null values again

appliances_df.isna().sum()

reviewText    324
summary       128
dtype: int64

In [8]:
# We can drop the rows with null values since they represent a very small percentage of data
# Also...can't do much with ones that don't have reviews.  

appliances_df.dropna(inplace = True)
appliances_df.isna().sum()

reviewText    0
summary       0
dtype: int64

### Cleaning Text

In [9]:
# for i in appliances_df['reviewText']:
#     print(i)

In [10]:
# Running several functions for cleaning text

appliances_df['reviewText'] = appliances_df['reviewText'].apply(lambda x: remove_url(x))
appliances_df['reviewText'] = appliances_df['reviewText'].apply(lambda x: remove_n(x))
appliances_df['reviewText'] = appliances_df['reviewText'].apply(lambda x: remove_emoji(x))

In [11]:
# Checking to make sure this worked

appliances_df['reviewText'][0]

"Not one thing in this book seemed an obvious original thought. However, the clarity with which this author explains how innovation happens is remarkable.Alan Gregerman discusses the meaning of human interactions and the kinds of situations that tend to inspire original and/or clear thinking that leads to innovation. These things include how people communicate in certain situations such as when they are outside of their normal patterns.Gregerman identifies the ingredients that make innovation more likely. This includes people being compelled to interact when they normally wouldn't, leading to serendipity. Sometimes the phenomenon will occur through collaboration, and sometimes by chance such as when an individual is away from home on travel.I recommend this book for its common sense, its truth and the apparent mastery of the subject by the author."

In [12]:
appliances_df['summary'] = clean_text(appliances_df['summary'])


In [13]:
appliances_df['summary'][0]

'Clear on what leads to innovation'

In [14]:
appliances_df.head()

,reviewText,summary
0,Not one thing in this book seemed an obvious o...,Clear on what leads to innovation
1,I have enjoyed Dr. Alan Gregerman's weekly blo...,Becoming more innovative by opening yourself t...
2,Alan Gregerman believes that innovation comes ...,The World from Different Perspectives
3,"Alan Gregerman is a smart, funny, entertaining...",Strangers are Your New Best Friends
4,"As I began to read this book, I was again remi...","How and why it is imperative to engage, learn ..."


### Text Preprocessing

#### Tokenizing

In [18]:
# Createa a function to tokenize 

def tokenize_words(text):
    # make everything lowercase 
    text = text.lower()
    
    # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # filter out stop words for now
    filtered_words = filter(lambda token: token not in stopwords.words('english'), tokens)
    
    return " ".join(filtered_words)

In [19]:
# Testing function

tokenize_words(appliances_df['reviewText'][0])

'one thing book seemed obvious original thought however clarity author explains innovation happens remarkable alan gregerman discusses meaning human interactions kinds situations tend inspire original clear thinking leads innovation things include people communicate certain situations outside normal patterns gregerman identifies ingredients make innovation likely includes people compelled interact normally leading serendipity sometimes phenomenon occur collaboration sometimes chance individual away home travel recommend book common sense truth apparent mastery subject author'

In [21]:
for review in appliances_df['reviewText']:
    review = tokenize_words(review)
    
appliances_df.head()

,reviewText,summary
0,Not one thing in this book seemed an obvious o...,Clear on what leads to innovation
1,I have enjoyed Dr. Alan Gregerman's weekly blo...,Becoming more innovative by opening yourself t...
2,Alan Gregerman believes that innovation comes ...,The World from Different Perspectives
3,"Alan Gregerman is a smart, funny, entertaining...",Strangers are Your New Best Friends
4,"As I began to read this book, I was again remi...","How and why it is imperative to engage, learn ..."


In [22]:
appliances_df['reviewText'] = appliances_df['reviewText'].apply(lambda x: tokenize_words(x))
appliances_df.head()

,reviewText,summary
0,one thing book seemed obvious original thought...,Clear on what leads to innovation
1,enjoyed dr alan gregerman weekly blog two prev...,Becoming more innovative by opening yourself t...
2,alan gregerman believes innovation comes viewi...,The World from Different Perspectives
3,alan gregerman smart funny entertaining writer...,Strangers are Your New Best Friends
4,began read book reminded deceptively complicat...,"How and why it is imperative to engage, learn ..."


## Data Exploration

### Basic NLP

## Modeling